In [ ]:
en-ru >> fr ready_res
ru-en >> fr

en-fr >>ru ready_res
fr-en >>ru ready_res

ru-fr>>en ready_res
fr-ru>>en ready_res

In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import os
import numpy as np

In [0]:
print(datetime.datetime.now().time())

13:01:41.294317


In [3]:
import datetime

In [2]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

import numpy as np
from tqdm import tqdm

In [0]:
from  sklearn.metrics.pairwise import cosine_similarity

In [0]:
%%time
from gensim.models.wrappers import FastText
model_fr = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')


In [0]:
%%time
from gensim.models.keyedvectors import FastTextKeyedVectors
model_ru= FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

CPU times: user 657 ms, sys: 3.79 s, total: 4.45 s
Wall time: 10.7 s


In [0]:
!pip3 install simpy.io

  Using cached https://files.pythonhosted.org/packages/3e/21/dfc98e6240f82c8507c1955aaad9f2649cf9fb351dbf4f46e2f33e9bf25e/simpy.io-0.2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/64/8f0fc71400d41b6c2c6443d333a1cade458d23d4945ccda700c810ff6aae/simpy-3.0.11-py2.py3-none-any.whl


In [0]:
from simpy.io import select as backend

In [0]:
%%time
#import io
def load_vectors(fname):
    fin = backend.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data
model_en = load_vectors("/Users/nigula/input/wiki-news-300d-1M.vec")

AttributeError: module 'simpy.io.select' has no attribute 'open'

In [26]:
def send_request_func(from_lang,to_lang, word, header_main, login, en_from_lang, en_to_lang):
    start = time.time()
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    print("going to find word in", url)
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    #
    save_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + en_from_lang + "_" + en_to_lang,word + ".xls")
    file = open(save_dir, "wb")
    file.write(response.content)
    file.close()
    print("saved to directory", save_dir)
    soup = bs(response.text, 'html.parser') 
    time.sleep(0.01)
    print("request_time",time.time() - start) 
    check_restiction(soup)
    return soup

In [25]:
def check_restiction(soup):
    for link in soup.find_all("section", attrs={"id" : "error-content"}):
        restrict = ['Your', 'access', 'is', 'temporarily', 'restricted']
        #print(link, "\\")
        for l in link.find_all("p"):
            count = 0 
            restict = False
            for word in l.text.strip().split()[:5]:
                #print(word, restrict[count])
                if word == restrict[count]:
                    restict = True
                else:
                    restict = False
                count += 1
            if restict == True:
                raise Exception('Restiction_limit!!!!!!!!!!!!!!')



In [0]:
a = ''
len(a)

0

In [18]:
def get_definitions_reverso(word, from_lang, to_lang, print_output = False):
    start = time.time()
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    senseword_list = []
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    try:
        word_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + land_dict[from_lang] + "_" + land_dict[to_lang],word + ".xls")
        infile = open(word_dir,"r")
        print("found_word_in", word_dir)
        contents = infile.read()
        soup = bs(contents,'html.parser')
    except:
        soup = send_request_func(from_lang,to_lang, word, header_main, login, land_dict[from_lang], land_dict[to_lang])
    #print(soup.prettify())
    first_string_passed = False
    pos = ''
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(sence_word)
        first_string_passed = True
        
    for link in soup.find_all("div", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            try:
                #print("||",link['data-pos'])
                pos = link['data-pos'][1:-1]
            except:
                pass
            sence_word = link.text.strip()
            senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(link.prettify())
    print("definiotns_time", time.time() - start)
    return senseword_list


get_definitions_reverso("одобрять", "русский", "французский", print_output = False)
#русский-французский/одобрять


found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/одобрять.xls
definiotns_time 0.08157706260681152


[{'sence_word': 'approuver', 'pos': 'v'},
 {'sence_word': 'entériner', 'pos': 'v'},
 {'sence_word': 'avaliser', 'pos': 'v'}]

In [6]:
get_definitions_reverso("brother", "английский", "русский", print_output = False)

found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_ru/brother.xls
definiotns_time 0.11357283592224121


[{'sence_word': 'брат', 'pos': 'nm'},
 {'sence_word': 'братец', 'pos': 'n'},
 {'sence_word': 'братишка', 'pos': 'n'},
 {'sence_word': 'братик', 'pos': 'n'},
 {'sence_word': 'братан', 'pos': 'n'},
 {'sence_word': 'братья', 'pos': 'n'},
 {'sence_word': 'дружище', 'pos': 'n'},
 {'sence_word': 'браток', 'pos': 'n'},
 {'sence_word': 'братские', 'pos': 'n'},
 {'sence_word': 'родной брат', 'pos': 'n'},
 {'sence_word': 'брательник', 'pos': 'n'},
 {'sence_word': 'братских', 'pos': 'n'}]

In [0]:
get_definitions_reverso("réchauffer", "французский", "русский", print_output = False)

going to find word in https://context.reverso.net/перевод/французский-русский/réchauffer
saved to directory /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_fr_ru/réchauffer.xls


[{'sence_word': 'согреть', 'pos': 'v'},
 {'sence_word': 'согреться', 'pos': 'v'},
 {'sence_word': 'разогреть', 'pos': 'v'},
 {'sence_word': 'подогреть', 'pos': 'v'}]

# LEMMATIZERS

In [7]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
def preprocess_words(words_list, lemmatizer_func):
    start = time.time()
    preprocessed_words = []
    for word in words_list:
        lemma = lemmatizer_func(word['sence_word'].lower())
        word['sence_word'] = lemma
    print("lemmat_time", time.time() - start)
        #print("lemma",lemma)
        #preprocessed_words.append(lemma)
    #return preprocessed_words
        

In [8]:

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            #print (lemma)
            try:
                orig_vector += model[lemma]
            except:
                pass
    return orig_vector


def en_lemmatize(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return ' '.join(lemmatized_line_list)

In [9]:
from pymystem3 import Mystem
text = "Красивая мама красиво мыла раму"
mystem = Mystem()
lemmas = mystem.lemmatize(text)
print(''.join(lemmas))

красивый мама красиво мыть рама



In [0]:
p = morph.parse('стали')[0]
p.normal_form

'стать'

In [11]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def ru_lemmatize(line):
    line_lemm = ''
    for word in line.split():
        lemma = morph.parse(word)[0]
        lemma = lemma.normal_form
        line_lemm += lemma + ' '
    line_lemm = line_lemm.strip()
    return line_lemm

ru_lemmatize("Красивая мама красиво мыла раму")

'красивый мама красиво мыло рам'

In [12]:
def ru_lemmatize(line):
    lemmas = mystem.lemmatize(line)
    return ''.join(lemmas)[:-1]
ru_lemmatize("Красивая мама красиво мыла раму")

'красивый мама красиво мыть рама'


# GOON

In [0]:
%%time
from gensim.models.wrappers import FastText
model_fr = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')


In [0]:
from gensim.models.keyedvectors import FastTextKeyedVectors
MOD= FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [0]:
MOD['мама']

array([ 0.02138138,  0.01521608,  0.00986125,  0.02197128,  0.06427927,
       -0.05983834, -0.01036269,  0.11541053,  0.03949647,  0.1092683 ,
       -0.03268374,  0.07753018, -0.01910948, -0.01419521, -0.03510416,
       -0.00682221, -0.04483933,  0.02470068, -0.09457379,  0.04353607,
       -0.05531164, -0.0661241 ,  0.08448623, -0.03727661, -0.0334726 ,
       -0.0091603 , -0.019973  , -0.08635031,  0.06308096, -0.01417614,
        0.02844894,  0.07066223,  0.03661515, -0.02241797,  0.00413776,
        0.02788121,  0.00193861, -0.03643694,  0.00609269,  0.00339793,
       -0.10118162, -0.06926611, -0.02972721,  0.06692997,  0.02539542,
       -0.01415602,  0.05340702,  0.00175135, -0.03582824,  0.01966216,
        0.0118806 ,  0.09134583,  0.01786702, -0.03293948,  0.03018913,
        0.04765303, -0.03240753,  0.03975621,  0.07407223, -0.10373922,
       -0.01794396,  0.01649946, -0.01978012,  0.0038766 , -0.02459781,
       -0.04664146, -0.01794693,  0.0562874 ,  0.00174485,  0.03

In [0]:
l1 = "maman"
l2 = 'maman famille'

def get_vectors_sum(lemm_line, lang):
    if lang == "французский": model = model_fr
    elif lang == "русский": model = MOD
    else:
        print("NO_MODEL_FOUND")
        return np.zeros(shape = 300)
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2, lang):
    l1_vect = get_vectors_sum(line_1, lang)
    l2_vect = get_vectors_sum(line_2, lang)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2, "французский")
#find_lines_similarity(l1, l2, "русский")



0.8403615321417536

In [13]:
def compare_pos(pos1, pos2):
    if pos1 == pos2:
        #print("abs eq", pos1, pos2)
        return True
    elif pos1 in pos2 or pos2 in pos1:
        #print("inclusive eq", pos1, pos2)
        return True
    #print("no eq", pos1, pos2)
    return False
def equate_empty_pos(pos1, pos2):
    if pos1 == '': 
        pos1 = pos2
        #print("pos1 eqauted")
    elif pos2 == '': 
        pos2 = pos1
        #print("pos2 eqauted")
    return pos1, pos2

In [14]:
def find_lines_inclusive_similarity(line_1,line_2):
    if line_1 in line_2: return True
    elif line_2 in line_1: return True
    else: return False
    
    
find_lines_inclusive_similarity("chauffer","réchauffer")

True

In [0]:
#handle indirect similarity
handled_words = []
for word_1 in list_1:
    for word_2 in list_2:
        word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
        pos_eq = compare_pos(word_1['pos'],word_2['pos'])
        if pos_eq == True and word_1['sence_word'] != word_2['sence_word'] and word_1['sence_word'] not in handled_words and word_2['sence_word'] not in handled_words:
            sim = find_lines_similarity(word_1['sence_word'], word_2['sence_word'], lists_lang)
            if sim > 0.9:
                print('COS_SIM_WORKED', word_1['sence_word'], word_2['sence_word'])
                if word_1['sence_word'] in cos_sim_dict:
                    handled_words.append(word_2['sence_word'])
                    cos_sim_dict[word_1['sence_word']].append(word_2['sence_word'])
                   # overall_intersection[word_1_index] += '| ' + word_2

                elif word_2['sence_word'] in cos_sim_dict:
                     handled_words.append(word_1['sence_word'])
                     cos_sim_dict[word_2['sence_word']].append(word_1['sence_word'])
                else:
                    cos_sim_dict[word_1['sence_word']] = [word_2['sence_word']]

In [0]:
time.time()

1571135656.759556

In [15]:
def get_two_target_lang_lists_intersection(list_1, list_2, lists_lang):
    start = time.time()
    overall_intersection = []
    cos_sim_dict ={}
    #overall_intersection
    for word_1 in list_1:
        for word_2 in list_2:
            #print(word_1, word_2)
            word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
            pos_eq = compare_pos(word_1['pos'],word_2['pos'])  
            if pos_eq == True and word_1['sence_word'] == word_2['sence_word']:
                #overall_intersection.append(word_1)
                cos_sim_dict[word_1['sence_word']] = []
    #print(cos_sim_dict)             
    #GETET INTERSECTION LIST
    for first_word in cos_sim_dict.keys():
        intersect_element = first_word
        for similar_word in cos_sim_dict[first_word]:
            intersect_element += " | " + similar_word
        overall_intersection.append(intersect_element)
    print(time.time() - start, "passed get_two_target_lang_lists_intersection")
    return overall_intersection

fr_proc_1 = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]

fr_proc_2 = [{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}]
get_two_target_lang_lists_intersection(fr_proc_1, fr_proc_2, "fr")      
            

7.700920104980469e-05 passed get_two_target_lang_lists_intersection


['réchauffer']

In [19]:
def get_cross_translate_reverso(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_dict = get_definitions_reverso(word_lang_1, lang_1, target_lang, print_output=True)
    preprocess_words(lang_1_to_target_words_dict, lemmatizer_func)
    
    lang_2_to_target_words_dict = get_definitions_reverso(word_lang_2, lang_2, target_lang, print_output=True)
    preprocess_words(lang_2_to_target_words_dict, lemmatizer_func)
    
    print("lang_1_to_target_words_dict", lang_1_to_target_words_dict,"\n")
    print("lang_2_to_target_words_dict", lang_2_to_target_words_dict)
    
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_to_target_words_dict, lang_2_to_target_words_dict,
                                                                 target_lang)
    
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate_reverso("warm", "английский", "согревать","русский", "французский", fr_lemmatize)


found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_fr/warm.xls
definiotns_time 0.10599303245544434
lemmat_time 6.198883056640625e-05
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/согревать.xls
definiotns_time 0.07021808624267578
lemmat_time 1.71661376953125e-05
lang_1_to_target_words_dict [{'sence_word': 'chaud', 'pos': 'adj'}, {'sence_word': 'chaleureux', 'pos': 'adj'}, {'sence_word': 'tiède', 'pos': 'adj'}, {'sence_word': 'doux', 'pos': 'adj'}, {'sence_word': 'bon', 'pos': 'adj/nm'}, {'sence_word': 'cordial', 'pos': 'adj'}, {'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chauffer', 'pos': 'v'}, {'sence_word': 'échauffer', 'pos': 'v'}, {'sence_word': 'se réchauffer', 'pos': 'v'}, {'sence_word': 'tiédir', 'pos': 'v'}, {'sence_word': 'chaleur', 'pos': 'nf'}, {'sence_word': 'accueil', 'pos': 'nm'}, {'sence_word': 'chaleureusement', 'pos': 'adv'}, {'sence_word': 'warm', 'pos': 'adv'}, {'sence_word': 'accueillante', 'pos': 'adv'}, {'sence_w

['réchauffer']

# en_ru_fr

In [0]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_new_vs_pos.csv")
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,before,до,preposition,['before the war'],['до войны']
1,1,before,впереди,preposition,[],[]
2,2,before,пред,preposition,[],[]
3,3,before,накануне,preposition,[],[]
4,4,before,передо,preposition,[],[]


In [20]:
def get_multilang_from_df(df, lookup_from, lookup_to, target_lang, target_lang_lemmatizer_func):
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    eng_word_list = []
    lang_1_word_list = []
    lang_2_word_list = []
    eng_word = 'first_word'
    save_triple_path = land_dict[lookup_from] + '_' + land_dict[lookup_to] + '_' + land_dict[target_lang]
    
    start = False
    
    #for df_ind in tqdm(range(len(df))):
    count = 0 
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if eng_word != row["word"]: print("\n************************************\n")
        eng_word = row["word"]
        #if eng_word == "trajet": start = True
        #if start == True:
        loc_1_word = row["local_word"]
        print(eng_word, loc_1_word)
        sense_intersect = get_cross_translate_reverso(eng_word, lookup_from, loc_1_word, lookup_to, target_lang, target_lang_lemmatizer_func)
        #sense_intersect = get_cross_translate_yand(eng_word, "en", loc_1_word,"ru", "fr", fr_lemmatize)
        print("\n SENSE_INTERSECTION ", sense_intersect)
        for sense_intersect_element in sense_intersect:
            eng_word_list.append(eng_word)
            lang_1_word_list.append(loc_1_word)
            lang_2_word_list.append(sense_intersect_element)


        if count % 10 == 0 and count != 0:

            save_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos",
                         save_triple_path, str(count) + ".csv")
            print(save_path)
            data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =[land_dict[lookup_from], land_dict[lookup_to], land_dict[target_lang]])
            data.to_csv(save_path)
        count += 1
            
    data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =[land_dict[lookup_from], land_dict[lookup_to], land_dict[target_lang]])
    
    return data

#df_din = get_multilang_from_df(df_en_ru_yandex, "английский", "русский", "французский", fr_lemmatize)
#df_din.head()

# EN_RU_FR

In [21]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_new_vs_pos_to_37353_no_freq.csv")
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,youngling,детеныш,noun,[],[]
1,1,youngling,юнец,noun,[],[]
2,2,asperity,лишения,noun,[],[]
3,3,asperity,шероховатость,noun,[],[]
4,4,asperity,резкость,noun,[],[]


In [22]:
df_en_ru_yandex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 6 columns):
Unnamed: 0        120000 non-null int64
word              120000 non-null object
local_word        120000 non-null object
pos               120000 non-null object
examples          120000 non-null object
local_examples    120000 non-null object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


In [24]:
df_en_ru_fr = get_multilang_from_df(df_en_ru_yandex, "английский", "русский", "французский", fr_lemmatize)


  0%|          | 0/120000 [00:00<?, ?it/s]


************************************

youngling детеныш
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_fr/youngling.xls


  0%|          | 1/120000 [00:00<12:52:04,  2.59it/s]

definiotns_time 0.24503612518310547
lemmat_time 1.9073486328125e-05
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/детеныш.xls
definiotns_time 0.08924221992492676
lemmat_time 1.4781951904296875e-05
lang_1_to_target_words_dict [{'sence_word': 'fillot', 'pos': 'n'}, {'sence_word': 'petit', 'pos': 'n'}, {'sence_word': 'jeunes jedi', 'pos': 'n'}] 

lang_2_to_target_words_dict [{'sence_word': 'bébé', 'pos': 'nm'}, {'sence_word': 'petit', 'pos': 'nm'}]
9.059906005859375e-06 passed get_two_target_lang_lists_intersection

 SENSE_INTERSECTION  ['petit']
youngling юнец
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_fr/youngling.xls
definiotns_time 0.08182191848754883
lemmat_time 1.6927719116210938e-05
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/юнец.xls


  0%|          | 2/120000 [00:00<10:36:19,  3.14it/s]

...

  0%|          | 43/120000 [00:26<16:50:18,  1.98it/s]

definiotns_time 0.08431100845336914
lemmat_time 5.3882598876953125e-05
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/простой.xls
definiotns_time 0.08381271362304688
lemmat_time 7.796287536621094e-05
lang_1_to_target_words_dict [{'sence_word': 'humble', 'pos': 'adj'}, {'sence_word': 'modeste', 'pos': 'adj'}, {'sence_word': 'simple', 'pos': 'adj'}, {'sence_word': 'humilier', 'pos': 'v'}, {'sence_word': 'humilité', 'pos': 'v'}, {'sence_word': 'modestie', 'pos': 'v'}, {'sence_word': 'humblement', 'pos': 'v'}, {'sence_word': 'pauvre', 'pos': 'v'}, {'sence_word': 'rabaisser', 'pos': 'v'}, {'sence_word': 'débonnaires', 'pos': 'v'}, {'sence_word': 'humiliez-vous', 'pos': 'v'}, {'sence_word': 'abaisser', 'pos': 'v'}, {'sence_word': 'modestement', 'pos': 'v'}, {'sence_word': 'malheureux', 'pos': 'v'}, {'sence_word': 'humilie-toi', 'pos': 'v'}] 

lang_2_to_target_words_dict [{'sence_word': 'simple', 'pos': 'adj'}, {'sence_word': 'facile', 'pos': 'adj'}, {'sence_word': 'ord

  0%|          | 44/120000 [00:26<18:15:54,  1.82it/s]

saved to directory /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/робкий.xls
request_time 0.5552480220794678
definiotns_time 0.5734162330627441
lemmat_time 2.3126602172851562e-05
lang_1_to_target_words_dict [{'sence_word': 'humble', 'pos': 'adj'}, {'sence_word': 'modeste', 'pos': 'adj'}, {'sence_word': 'simple', 'pos': 'adj'}, {'sence_word': 'humilier', 'pos': 'v'}, {'sence_word': 'humilité', 'pos': 'v'}, {'sence_word': 'modestie', 'pos': 'v'}, {'sence_word': 'humblement', 'pos': 'v'}, {'sence_word': 'pauvre', 'pos': 'v'}, {'sence_word': 'rabaisser', 'pos': 'v'}, {'sence_word': 'débonnaires', 'pos': 'v'}, {'sence_word': 'humiliez-vous', 'pos': 'v'}, {'sence_word': 'abaisser', 'pos': 'v'}, {'sence_word': 'modestement', 'pos': 'v'}, {'sence_word': 'malheureux', 'pos': 'v'}, {'sence_word': 'humilie-toi', 'pos': 'v'}] 

lang_2_to_target_words_dict [{'sence_word': 'timide', 'pos': 'adj'}, {'sence_word': 'craintif', 'pos': 'adj'}]
2.8133392333984375e-05 passed get_two_target_lang_li

KeyboardInterrupt: 

In [0]:
df_en_ru_fr.head()

,en,ru,fr
0,before,до,avant
1,before,до,auparavant
2,before,до,précédant
3,before,впереди,devant
4,before,пред,devant


In [0]:
df_en_ru_fr.to_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/en_ru_fr_no_cossim.csv")

# FR_EN_RU

In [0]:
df_fr_en_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_en.csv")
df_fr_en_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,tomber,fall,verb,[],[]
1,1,tomber,fell,noun,[],[]
2,2,tomber,fall down,verb,[],[]
3,3,pénétrer,penetrate,verb,[],[]
4,4,complet,full,noun,[],[]


In [0]:
df_fr_en_yandex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 6 columns):
Unnamed: 0        1048 non-null int64
word              1048 non-null object
local_word        1048 non-null object
pos               1048 non-null object
examples          1048 non-null object
local_examples    1048 non-null object
dtypes: int64(1), object(5)
memory usage: 49.2+ KB


In [0]:
df_fr_en_ru = get_multilang_from_df(df_fr_en_yandex, "французский", "английский", "русский", ru_lemmatize)


In [0]:
df_fr_en_ru.to_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/fr_en_ru_no_cossim.csv")



In [0]:
df_fr_en_ru.head(50)

,fr,en,ru
0,tomber,fall,упасть
1,tomber,fall,падать
2,tomber,fall,пасть
3,tomber,fall,оказаться
4,tomber,fell,упасть
5,tomber,fell,пасть
6,tomber,fall down,упасть
7,tomber,fall down,падать
8,pénétrer,penetrate,проникнуть
9,pénétrer,penetrate,проникать


# EN_FR_RU

In [0]:
df_en_fr_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_fr_new_vs_pos.csv")
df_en_fr_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,before,avant,preposition,[],[]
1,1,before,auparavant,adverb,[],[]
2,2,adverb,adverbe,noun,[],[]
3,3,during,lors,adverb,[],[]
4,4,during,durant,preposition,[],[]


In [0]:
del df_en_fr_yandex['Unnamed: 0']
df_en_fr_ru = get_multilang_from_df(df_en_fr_yandex, "английский", "французский", "русский", ru_lemmatize)


In [0]:
df_en_fr_ru.to_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/en_fr_ru_no_cossim.csv")


# FR_RU_EN

In [3]:
def get_reverse_dict(df_path, lookup_from, lookup_to, target, lemmatizer_func):
    df = pd.read_csv(df_path)
    try:
        del df['Unnamed: 0']
    except:
        pass
    df_crosslang = get_multilang_from_df(df, lookup_from, lookup_to, target, lemmatizer_func)
    return df_crosslang
               


In [ ]:
df_fr_ru_en = get_reverse_dict("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_ru.csv",
                     "французский", "русский","английский", en_lemmatize)

In [0]:
df_fr_ru_en_full_only = df_fr_ru_en[df_fr_ru_en['en'] != '']
df_fr_ru_en_full_only.head(20)

,fr,ru,en
0,tomber,упасть,fall
1,tomber,упасть,drop
2,tomber,упасть,fall down
3,tomber,попасть,fall into
5,tomber,опускать,drop out
6,tomber,опадать,no_equality
8,tomber,снижаться,fall
9,tomber,снижаться,drop
10,tomber,снижаться,go down
11,tomber,снижаться,come down


In [0]:
df_fr_ru_en_full_only.to_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/fr_ru_en_no_cossim.csv")


In [0]:
df_fr_ru_en_full_only

In [0]:
df_fr_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_ru.csv")
df_fr_ru_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,tomber,упасть,verb,"['branches tombées', 'tomber à terre', 'tomber...","['упавшие ветви', 'падать на землю', 'пасть же..."
1,1,tomber,попасть,verb,['tomber en disgrâce'],['попадать в немилость']
2,2,tomber,опускать,verb,[],[]
3,3,tomber,опадать,verb,[],[]
4,4,tomber,положить,verb,[],[]


In [0]:
del df_fr_ru_yandex['Unnamed: 0']

# RU_FR_EN

In [0]:
df_ru_fr_en = get_reverse_dict("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_ru_fr.csv",
                     "русский", "французский","английский", ru_lemmatize)

In [0]:
df_ru_fr_en.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_ru_fr_en.csv")

# RU_EN_FR

df_ru_en_fr = get_reverse_dict("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_ru_en_fr.csv", "русский",
                                     "английский", "французский", fr_lemmatize )